In [12]:
from datetime import datetime as dt

start = dt.now()

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from prophet import Prophet
plt.style.use('ggplot')
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout,GRU
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from keras.models import Sequential
from sklearn import preprocessing
from keras.layers import Dense, Activation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from keras.optimizers import RMSprop

In [14]:
def data():
    df = pd.read_csv("time_series_covid19_confirmed_global.csv")
    df.set_index('Country/Region', inplace=True)
    india_data = df.loc['India']
    india_data = india_data.drop(['Province/State', 'Lat', 'Long'])
    india_data = india_data.transpose()
    india_data.columns = ["Total Confirmed"]
    data = pd.DataFrame(india_data)
    data['India'] = pd.to_numeric(data['India'], errors='coerce')
    data1 = data.rename(columns={'India': 'Total Confirmed'})
    return data1

In [15]:
data().head()

,Total Confirmed
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0
1/26/20,0


In [16]:
def arima(data1):
    train_len = (int)(0.8*len(data1))
    df_train = data1.iloc[:train_len]
    df_test = data1.iloc[train_len:]
    model=ARIMA(df_train,order=(2,2,2))
    model_fit=model.fit()
    res=model_fit.predict(start = len(df_train), end = 1143, dynamic=True)
    res = pd.DataFrame(res)
    res.index = pd.to_datetime(res.index)
    df_test.index = pd.to_datetime(df_test.index)
    data1.index = pd.to_datetime(data1.index)
    res = res[:-1]
    MAPE = mean_absolute_percentage_error(df_test['Total Confirmed'].values, res['predicted_mean'].values)*100
    mdape = np.median((np.abs(np.subtract(df_test['Total Confirmed'].values, res['predicted_mean'].values)/df_test['Total Confirmed'].values)))*100
    smape = 100/len(df_test['Total Confirmed'].values) * np.sum(2 * np.abs(df_test['Total Confirmed'].values - res['predicted_mean'].values) / (np.abs(df_test['Total Confirmed'].values) + np.abs(res['predicted_mean'].values)))
    print("ARIMA MAPE: {:.4f}".format(MAPE))
    print("ARIMA MDAPE: {:.4f}".format(mdape))
    print('ARIMA SMAPE: {:.4f}'.format(smape))
    df = pd.read_csv('graph.csv')
    df['ARIMA'] = res['predicted_mean'].values
    df.to_csv('graph.csv', index=False)
arima(data())


ARIMA MAPE: 3.7593
ARIMA MDAPE: 3.5425
ARIMA SMAPE: 3.6546


In [17]:
def fit_model(model, X_train, y_train, X_val, y_val):
    early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),
                        batch_size=16, shuffle=False, callbacks=[early_stop])
    return history

def create_dataset (X, look_back = 1):
  Xs,ys = [],[]
  for i in range(len(X)-look_back):
    v = X[i:i+look_back]
    Xs.append(v)
    ys.append(X[i+look_back])
  return np.array(Xs),np.array(ys)

def prediction(model,scaler,X_test):
  prediction = model.predict(X_test)
  prediction = prediction.reshape(-1, prediction.shape[-1])
  prediction = scaler.inverse_transform(prediction)
  return prediction

def evaluate_prediction(predictions,actual,mod):
  mape = mean_absolute_percentage_error(actual, predictions)*100
  mdape = np.median((np.abs(np.subtract(actual, predictions)/ actual)))*100
  smape = 100/len(actual) * np.sum(2 * np.abs(predictions-actual) / (np.abs(actual) + np.abs(predictions)))
  print(mod," MAPE: {:.4f}".format(mape))
  print(mod, " MDAPE: {:.4f}".format(mdape))
  print(mod, " SMAPE: {:.4f}".format(smape))

def lst(df):
    train_length = (int)(len(df)*0.8)
    train = df[:train_length-1]
    test = df[train_length-1:]
    # print(len(test))
    scaler = MinMaxScaler().fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    # print(len(test_scaled))
    X_train,y_train = create_dataset(train_scaled,look_back = 1)
    X_test,y_test = create_dataset(test_scaled,look_back = 1)
    # print(len(y_test))
    X_train, X_val = X_train[:int(len(X_train)*0.9)], X_train[int(len(X_train)*0.9):]
    y_train, y_val = y_train[:int(len(y_train)*0.9)], y_train[int(len(y_train)*0.9):]
    model_lstm = Sequential()
    model_lstm.add(LSTM(64, return_sequences = True, activation='relu', input_shape= [X_train.shape[1],X_train.shape[2]]))
    model_lstm.add(Dense(1))
    model_lstm.compile(optimizer='adam', loss='mse')
    history_lstm = fit_model(model_lstm, X_train, y_train, X_val, y_val)
    y_test = scaler.inverse_transform(y_test)
    y_train = scaler.inverse_transform(y_train)
    prediction_lstm = prediction(model_lstm, scaler, X_test)
    evaluate_prediction(prediction_lstm,y_test,"LSTM")
    # print(len(prediction_lstm))
    df = pd.read_csv('graph.csv')
    df['LSTM'] = prediction_lstm
    df.to_csv('graph.csv', index=False)
lst(data())

Epoch 1/100
52/52 [==============================] - 2s 7ms/step - loss: 0.1939 - val_loss: 0.5464
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1063 - val_loss: 0.3028
Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0603 - val_loss: 0.1424
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0447 - val_loss: 0.0662
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0396 - val_loss: 0.0361
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0339 - val_loss: 0.0218
Epoch 7/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0273 - val_loss: 0.0131
Epoch 8/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0209 - val_loss: 0.0071
Epoch 9/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0154 - val_loss: 0.0033
Epoch 10/100
52/52 [==============================] - 0s 2ms/step - loss: 0.0108 - val_loss: 0.0011
Epoch 11/

In [18]:
def fit_model(model, X_train, y_train, X_val, y_val):
    early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val),
                        batch_size=16, shuffle=False, callbacks=[early_stop])
    return history

def gru_model(df):
    train_length = (int)(len(df)*0.8)
    train = df[:train_length-1]
    test = df[train_length-1:]
    scaler = MinMaxScaler().fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    X_train,y_train = create_dataset(train_scaled,look_back = 1)
    X_test,y_test = create_dataset(test_scaled,look_back = 1)
    X_train, X_val = X_train[:int(len(X_train)*0.9)], X_train[int(len(X_train)*0.9):]
    y_train, y_val = y_train[:int(len(y_train)*0.9)], y_train[int(len(y_train)*0.9):]
    model_gru = Sequential()
    model_gru.add(GRU(units = 64,return_sequences = True,
                input_shape = [X_train.shape[1],X_train.shape[2]]))
    model_gru.add(Dropout(0.2))
    model_gru.add(GRU(units = 64))
    model_gru.add(Dropout(0.2))
    model_gru.add(Dense(units = 1))
    model_gru.compile(optimizer = "adam",loss = "mse")
    history_gru = fit_model(model_gru, X_train, y_train, X_val, y_val)
    y_test = scaler.inverse_transform(y_test)
    y_train = scaler.inverse_transform(y_train)
    prediction_gru = prediction(model_gru,scaler, X_test)
    evaluate_prediction(prediction_gru,y_test,"GRU")
    df = pd.read_csv('graph.csv')
    df['GRU'] = prediction_gru
    df.to_csv('graph.csv', index=False)
gru_model(data())


Epoch 1/100
52/52 [==============================] - 5s 19ms/step - loss: 0.0097 - val_loss: 0.0028
Epoch 2/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0605 - val_loss: 0.0067
Epoch 3/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0237 - val_loss: 0.0045
Epoch 4/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0089 - val_loss: 5.8198e-05
Epoch 5/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0032 - val_loss: 2.3746e-04
Epoch 6/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0025 - val_loss: 1.9107e-04
Epoch 7/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0025 - val_loss: 1.6419e-06
Epoch 8/100
52/52 [==============================] - 0s 5ms/step - loss: 0.0028 - val_loss: 3.0841e-04
Epoch 9/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0030 - val_loss: 4.2332e-04
Epoch 10/100
52/52 [==============================] - 0s 4ms/step - loss: 0.0027 - v

In [19]:
def prophet_model(data, skillset):
    prophet_Data = data[[skillset]]
    train_size = (int)(0.8*len(data))
    new_percentage = train_size
    train_prophet_Data = prophet_Data.head(new_percentage)
    test_prophet_Data = prophet_Data.iloc[new_percentage:len(prophet_Data),:]
    ts1 = pd.DataFrame({'ds': train_prophet_Data.index,"y": train_prophet_Data[skillset]})
    prophet = Prophet()
    print(ts1.shape)
    prophet.fit(ts1)
    future = prophet.make_future_dataframe(periods = len(data)-new_percentage, freq = 'D')
    forecast = prophet.predict(future)
    forecasted_values = forecast[['ds','yhat', 'yhat_lower','yhat_upper']]
    print(forecasted_values.shape)
    prophet_df = pd.DataFrame(forecasted_values[["yhat"]])
    a = list(prophet_Data.index)
    print(len(a))
    print(prophet_df.shape)
    prophet_df1 = prophet_df.set_axis(a)
    prophet_df1.rename(columns = {list(prophet_df1)[0]: "prophet_prediction"}, inplace=True)
    print(prophet_df1.head())
    final_data = pd.concat([prophet_Data[[skillset]],prophet_df1['prophet_prediction'].apply(np.floor)],axis=1)
    print(final_data.head())
    prophet_percent = train_size
    prophet_metric_train = final_data.head(prophet_percent)
    prophet_metric_test = final_data.iloc[prophet_percent: len(final_data),:]
    print(prophet_metric_test.head())
    print(prophet_metric_test.shape)
    MAPE = mean_absolute_percentage_error(prophet_metric_test[skillset].values, prophet_metric_test["prophet_prediction"].values)*100
    mdape = np.median((np.abs(np.subtract(prophet_metric_test[skillset].values, prophet_metric_test["prophet_prediction"].values)/prophet_metric_test[skillset].values)))*100
    smape = 100/len(prophet_metric_test[skillset].values) * np.sum(2 * np.abs(prophet_metric_test[skillset].values - prophet_metric_test["prophet_prediction"].values) / (np.abs(prophet_metric_test[skillset].values) + np.abs(prophet_metric_test["prophet_prediction"].values)))
    print("Prophet MAPE: {:.4f}".format(MAPE))
    print("Prophet MDAPE: {:.4f}".format(mdape))
    print('Prophet SMAPE: {:.4f}'.format(smape))
    df = pd.read_csv('graph.csv')
    df['prophet'] = prophet_metric_test["prophet_prediction"].values
    df.to_csv('graph.csv', index=False)
prophet_model(data(), 'Total Confirmed')

(914, 2)


21:58:28 - cmdstanpy - INFO - Chain [1] start processing
21:58:28 - cmdstanpy - INFO - Chain [1] done processing


(1143, 4)
1143
(1143, 1)
         prophet_prediction
1/22/20      -827217.038260
1/23/20      -680664.845781
1/24/20      -539671.319792
1/25/20      -405410.222607
1/26/20      -279644.662286
         Total Confirmed  prophet_prediction
1/22/20                0           -827218.0
1/23/20                0           -680665.0
1/24/20                0           -539672.0
1/25/20                0           -405411.0
1/26/20                0           -279645.0
         Total Confirmed  prophet_prediction
7/24/22         43905621          44003712.0
7/25/22         43920451          44035211.0
7/26/22         43938764          44072042.0
7/27/22         43959321          44121218.0
7/28/22         43979730          44163922.0
(229, 2)
Prophet MAPE: 15.4722
Prophet MDAPE: 14.2306
Prophet SMAPE: 13.8885


In [20]:
def arima_ann(data1):
    len_training = int(len(data1)*.8)
    df_train,df_test = data1[:len_training], data1[len_training:]
    predicted_list, error_list = [], []
    model = ARIMA(df_train, order=(2, 2, 2))
    model_fit = model.fit()
    predicted_values = model_fit.predict(start = len(df_train), end = 1143, dynamic=True)
    for t in range(len(df_test)):
        obs = df_test.iloc[t]['Total Confirmed']
        predicted_value = predicted_values[t]
        error = abs(predicted_value - obs)
        error_list.append(obs - predicted_value)
        predicted_list.append(predicted_value)
    training_error = model_fit.resid
    model = Sequential()
    model.add(Dense(100, input_dim=1, kernel_initializer="uniform", activation="tanh"))
    model.add(Dense(1, kernel_initializer="uniform"))
    model.add(Activation("linear"))
    optimizer = RMSprop(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    train_X,train_Y = [],[]
    for i in range(0 , len(training_error) - 1):
        train_X.append(training_error[i:i+1])
        train_Y.append(training_error[i+1])
    new_train_X,new_train_Y = [],[]
    for i in train_X:
        new_train_X.append(i)
    for i in train_Y:
        new_train_Y.append(i)
    new_train_X = np.array(new_train_X)
    new_train_Y = np.array(new_train_Y)

    scaler_X = preprocessing.MinMaxScaler()
    scaler_Y = preprocessing.MinMaxScaler()
    new_train_X_scaled = scaler_X.fit_transform(new_train_X.reshape(-1,1))
    new_train_Y_scaled = scaler_Y.fit_transform(new_train_Y.reshape(-1,1))

    new_train_X_scaled, X_val = new_train_X_scaled[:int(len(new_train_X_scaled)*0.9)], new_train_X_scaled[int(len(new_train_X_scaled)*0.9):]
    new_train_Y_scaled, y_val = new_train_Y_scaled[:int(len(new_train_Y_scaled)*0.9)], new_train_Y_scaled[int(len(new_train_Y_scaled)*0.9):]

    
    early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    model.fit(new_train_X_scaled, new_train_Y_scaled, epochs=100, validation_data=(X_val, y_val), batch_size=10, verbose=1, callbacks=[early_stop])
    test_data = []
    for i in error_list:
        test_data.append(i)
    test_data = np.array(test_data)
    scaler_test = preprocessing.MinMaxScaler()
    test_data_scaled = scaler_test.fit_transform(test_data.reshape(-1,1))
    predictions_scaled = model.predict(test_data_scaled)
    predictions = scaler_test.inverse_transform(predictions_scaled)
    pred_final = predictions + predicted_list
    mape = mean_absolute_percentage_error(df_test['Total Confirmed'].to_list(),pred_final[0])
    print('Test MAPE: %f' % mape)
    evaluate_prediction(pred_final[0],df_test['Total Confirmed'].to_list(),"ARIMA+ANN")
    df = pd.read_csv('graph.csv')
    df['Hybrid ARIMA+ANN'] = pred_final[0]
    df.to_csv('graph.csv', index=False)
arima_ann(data())

Epoch 1/100
83/83 [==============================] - 1s 3ms/step - loss: 0.0511 - val_loss: 0.0015
Epoch 2/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0089 - val_loss: 0.0019
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0089 - val_loss: 5.8322e-04
Epoch 4/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0085 - val_loss: 5.8756e-04
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0086 - val_loss: 0.0035
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0085 - val_loss: 5.5649e-04
Epoch 7/100
83/83 [==============================] - 0s 1ms/step - loss: 0.0084 - val_loss: 5.3542e-04
Epoch 8/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0083 - val_loss: 5.4225e-04
Epoch 9/100
83/83 [==============================] - 0s 1ms/step - loss: 0.0082 - val_loss: 0.0023
Epoch 10/100
83/83 [==============================] - 0s 2ms/step - loss: 0.0083 - val_lo

In [21]:
running_secs = (dt.now() -start).seconds

In [22]:
running_secs

23